Расмотрим пример на датасете из репозитория UCI

Описание данных - https://archive.ics.uci.edu/ml/datasets/banknote+authentication#

In [93]:
import pandas as pd
import numpy as np
data = pd.read_csv('adult.data')
data.head(3)

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K


У нас есть 4 признака и 1 целевая переменная (бинарная) - нужно определить поддельная купюра или нет

In [94]:
pd.DataFrame([data.isna().sum(), data.isnull().sum()], index={'na', 'null'})

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
na,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
null,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [95]:
print(data.shape)

(32561, 15)


Всего 1372 купюры

Посмотрим на соотношение классов

In [96]:
data.iloc[:, -1].value_counts()

 <=50K    24720
 >50K      7841
Name: income, dtype: int64

In [97]:
data['income'] = data['income'].map({' <=50K': 0, ' >50K': 1})

Разбиваем выборку на тренировочную и тестовую части и обучаем модель (в примере - градиентный бустинг)

In [98]:
from sklearn.model_selection import train_test_split

x_data = pd.get_dummies(data.iloc[:, :-1])
y_data = data['income']

x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.2, random_state=7)

In [99]:
import xgboost as xgb

model = xgb.XGBClassifier()

model.fit(x_train, y_train)
y_predict = model.predict(x_test)

/home/ivan/.local/lib/python3.8/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[17:13:32] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


Проверяем качество

In [100]:
from sklearn.metrics import recall_score, precision_score, roc_auc_score, accuracy_score, f1_score

def evaluate_results(y_test, y_predict):
    print('Classification results:')
    f1 = f1_score(y_test, y_predict)
    print("f1: %.2f%%" % (f1 * 100.0)) 
    roc = roc_auc_score(y_test, y_predict)
    print("roc: %.2f%%" % (roc * 100.0)) 
    rec = recall_score(y_test, y_predict, average='binary')
    print("recall: %.2f%%" % (rec * 100.0)) 
    prc = precision_score(y_test, y_predict, average='binary')
    print("precision: %.2f%%" % (prc * 100.0)) 
    return f1, prc, rec, roc

    
evaluate_results(y_test, y_predict)

Classification results:
f1: 70.90%
roc: 79.60%
recall: 64.98%
precision: 78.02%


(0.7090464547677262,
 0.7801691006917756,
 0.6498079385403329,
 0.7960209153416673)

### Теперь очередь за PU learning

Представим, что нам неизвестны негативы и часть позитивов

In [101]:
mod_data = pd.get_dummies(data.copy())
#get the indices of the positives samples
pos_ind = np.where(mod_data.loc[:, 'income'].values == 1)[0]
#shuffle them
np.random.shuffle(pos_ind)
# leave just 25% of the positives marked
pos_sample_len = int(np.ceil(0.25 * len(pos_ind)))
print(f'Using {pos_sample_len}/{len(pos_ind)} as positives and unlabeling the rest')
pos_sample = pos_ind[:pos_sample_len]

Using 1961/7841 as positives and unlabeling the rest


Создаем столбец для новой целевой переменной, где у нас два класса - P (1) и U (-1)

In [102]:
mod_data['class_test'] = -1
mod_data.loc[pos_sample, 'class_test'] = 1
print('target variable:\n', mod_data.iloc[:, -1].value_counts())

target variable:
 -1    30600
 1     1961
Name: class_test, dtype: int64


* We now have just 153 positive samples labeled as 1 in the 'class_test' col while the rest is unlabeled as -1.

* Recall that col 4 still holds the actual label

In [103]:
mod_data.head(10)

,age,fnlwgt,education-num,capital-gain,capital-loss,hours-per-week,income,workclass_ ?,workclass_ Federal-gov,workclass_ Local-gov,...,native-country_ Puerto-Rico,native-country_ Scotland,native-country_ South,native-country_ Taiwan,native-country_ Thailand,native-country_ Trinadad&Tobago,native-country_ United-States,native-country_ Vietnam,native-country_ Yugoslavia,class_test
0,39,77516,13,2174,0,40,0,0,0,0,...,0,0,0,0,0,0,1,0,0,-1
1,50,83311,13,0,0,13,0,0,0,0,...,0,0,0,0,0,0,1,0,0,-1
2,38,215646,9,0,0,40,0,0,0,0,...,0,0,0,0,0,0,1,0,0,-1
3,53,234721,7,0,0,40,0,0,0,0,...,0,0,0,0,0,0,1,0,0,-1
4,28,338409,13,0,0,40,0,0,0,0,...,0,0,0,0,0,0,0,0,0,-1
5,37,284582,14,0,0,40,0,0,0,0,...,0,0,0,0,0,0,1,0,0,-1
6,49,160187,5,0,0,16,0,0,0,0,...,0,0,0,0,0,0,0,0,0,-1
7,52,209642,9,0,0,45,1,0,0,0,...,0,0,0,0,0,0,1,0,0,-1
8,31,45781,14,14084,0,50,1,0,0,0,...,0,0,0,0,0,0,1,0,0,-1
9,42,159449,13,5178,0,40,1,0,0,0,...,0,0,0,0,0,0,1,0,0,-1


Remember that this data frame (x_data) includes the former target variable that we keep here just to compare the results

[:-2] is the original class label for positive and negative data [:-1] is the new class for positive and unlabeled data

In [104]:
x_data = mod_data.drop(['income', 'class_test'], axis=1).values  # just the X
y_labeled = mod_data.loc[:, 'class_test'].values  # new class (just the P & U)
y_positive = mod_data.loc[:, 'income'].values  # original class

### 1. random negative sampling

In [105]:
mod_data = mod_data.sample(frac=1)
neg_sample = mod_data[mod_data['class_test'] == -1][:len(mod_data[mod_data['class_test'] == 1])]
sample_test = mod_data[mod_data['class_test'] == -1][len(mod_data[mod_data['class_test'] == 1]):]
pos_sample = mod_data[mod_data['class_test'] == 1]
print(neg_sample.shape, pos_sample.shape)
sample_train = pd.concat([neg_sample, pos_sample]).sample(frac=1)

(1961, 110) (1961, 110)


In [106]:
model_rns = xgb.XGBClassifier()

model.fit(sample_train.drop(['income', 'class_test'], axis=1).values,
          sample_train.loc[:, 'income'].values)
y_predict_rns = model.predict(sample_test.drop(['income', 'class_test'], axis=1).values)

evaluate_results(sample_test.loc[:, 'income'].values, y_predict_rns)

[17:13:35] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/home/ivan/.local/lib/python3.8/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


Classification results:
f1: 60.95%
roc: 81.98%
recall: 87.76%
precision: 46.69%


(0.6095141828289847,
 0.46689895470383275,
 0.8775695834091323,
 0.8197803841339153)

In [107]:
pd.DataFrame([
    evaluate_results(y_test, y_predict),
    evaluate_results(sample_test.loc[:, 'income'].values, y_predict_rns)
], columns={'f1', 'precision', 'recall', 'roc'}, index={'Simple xgboost', 'random negative sampling'}).round(2)

Classification results:
f1: 70.90%
roc: 79.60%
recall: 64.98%
precision: 78.02%
Classification results:
f1: 60.95%
roc: 81.98%
recall: 87.76%
precision: 46.69%


,recall,f1,precision,roc
Simple xgboost,0.71,0.78,0.65,0.80
random negative sampling,0.61,0.47,0.88,0.82


### Домашнее задание

1. взять любой набор данных для бинарной классификации (можно скачать один из модельных с https://archive.ics.uci.edu/ml/datasets.php)
3. сделать feature engineering
4. обучить любой классификатор (какой вам нравится)
5. далее разделить ваш набор данных на два множества: P (positives) и U (unlabeled). Причем брать нужно не все положительные (класс 1) примеры, а только лишь часть
6. применить random negative sampling для построения классификатора в новых условиях
7. сравнить качество с решением из пункта 4 (построить отчет - таблицу метрик)
8. поэкспериментировать с долей P на шаге 5 (как будет меняться качество модели при уменьшении/увеличении размера P)

<b>Бонусный вопрос:</b>

Как вы думаете, какой из методов на практике является более предпочтительным: random negative sampling или 2-step approach?

Ваш ответ здесь: